# **7. SIMILARITY MEASURES AND DUPLICATE DETECTION**

**PART 1: SIMILARITIES MEASURES**

- Distance-Based Comparison Functions

  - String-Based Distance Functions
    - Edit-based Similarity Measures
      - Edit Distance/Levenshtein Distance
    - Phonetic-based Algorithms
      - Sounded Code (American Soundex)
    - Jaro-Winkler String Comparator
  - Item-Based Distance Functions
    - Jaccard Distance
    - TF-IDF: Term Frequency - Inverse Document Frequency (or Cosine Similarity)

Import libraries:

In [ ]:
!pip install python-Levenshtein pyphonetics jaro-winkler recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import Levenshtein as lev
import pyphonetics
from pyphonetics import Soundex
import jaro
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import recordlinkage
from recordlinkage.datasets import load_febrl1

**1. Edit-based** *Edit Distance/Levenstein Distance*

  Minimum number of edits from one word to the other.

        Similarity = 1 - Levenshtein Distance/Max Length
        Levenstein Ratio = Levenstein Distance/Alignment Length

In [ ]:
def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1

    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1,
               LD(s[:-1], t[:-1]) + cost])
    return res

In [ ]:
#Distance computation
name1 = "Maria"
name2 = "Mariella"
L_distance = (LD(name1, name2))
L_distance

3

In [ ]:
#Maximum length
ML = max(len(name1), len(name2))
ML

5

In [ ]:
#Similarity evaluation = 1 - Levenshtein Distance/Max Length
L_similarirty = (1-(L_distance/ML))
L_similarirty

0.4

In [ ]:
#There is also a library
L_distance = lev.distance(name1,name2)
L_distance

3

In [ ]:
#Compute the lev ratio = Levenshtein Distance/Alignment Length
L_ratio = lev.ratio(name1,name2)
L_ratio

0.7692307692307692

**2. Phonetic-based** *American Soundex Algorithm*

These algorithms can find similar sounding terms and names.

      1. The first character of the word is retained as the first character of the Soundex code.
      2. The following letters are discarded: a,e,i,o,u,h,w, and y.
      3. Remaining consonants are given a code number.
      4. If consonants having the same code number appear consecutively, the number will only be coded once. (e.g. "B233" becomes "B23")
      5. The resulting code is modified so that it becomes exactly four characters long: if it is less than 4 characters, zeroes are added to the end (e.g. "B2" becomes "B200")
      6. If it is more than 4 characters, the code is truncated (e.g. "B2435" becomes "B243")


In [ ]:
help(pyphonetics)

Help on package pyphonetics:

NAME
    pyphonetics - A Python 3 phonetics library.

PACKAGE CONTENTS
    distance_metrics (package)
    exceptions
    phonetics (package)
    utils

VERSION
    0.5.3

FILE
    /usr/local/lib/python3.10/dist-packages/pyphonetics/__init__.py




In [ ]:
soundex = Soundex()

In [ ]:
#Sounded code of Maria
soundex.phonetics('Maria')

'M600'

In [ ]:
#Sounded code of Mariella
soundex.phonetics('Mariella')

'M640'

In [ ]:
soundex.sounds_like('Maria', 'Mariella')

False

In [ ]:
soundex.sounds_like('London', 'Londonn')

True

In [ ]:
#Compute the Levenshtein distance between Soundex Codes
soundex.distance('Maria', 'Mariella', metric='levenshtein')

1

**3. Jaro-Winkler** *Similarity Index*

The Jaro-Winkler string comparator counts:

  - the number c of matching characters between two strings
  (limited to the greatest integer of half the length of the longer string --> c <= max(len(s1),len(s2))/2 )
  - the number of transpositions t that are the number of matching characters that are not in the right order divided by two
  - n is the length of string 1
  - m is the length of string 2

        Jaro-Similarity = 1/3 * (c/m + c/n + (c-t)/c)

        Jaro–Winkler Similarity = sim(j) + lp(1 - sim(j)) where:
          -  sim(j) is the Jaro-Similarity for strings s1 and s2
          - l is the length of common prefix at the start of the string up to a maximum of 4 characters
          - p is a constant scaling factor for how much the score is adjusted upwards for having common prefixes

In [ ]:
jaro.jaro_winkler_metric('Maria', 'Mariella')

0.925

In [ ]:
help(jaro)

Help on package jaro:

NAME
    jaro - Python translation of the original Jaro-Winkler functions.

DESCRIPTION
    The Jaro-Winkler functions compare two strings and return a score indicating
    how closely the strings match. The score ranges from 0 (no match) to 1
    (perfect match).
    
    Two null strings ('') will compare as equal. Strings should be unicode
    strings, and will be compared as given; the caller is responsible for
    capitalisations and trimming leading/trailing spaces.
    
    You should normally only need to use either the jaro_metric() or
    jaro_winkler_metric() functions defined here. If you want to implement your
    own, non-standard metrics, look at the comments and functions in the jaro.py
    submodule.
    
    The C-source code containing the original functions was found here:
    
    http://web.archive.org/web/20100227020019/http://www.census.gov/geo/msb/stand/strcmp.c
    
    This module should output exactly the same numbers as that code, for

**4. Item-based** *Jaccard Similarity*

Jaccard similarity index is defined as the intersection of the two sets of words dividing by the union of them.

- Let A and B the set of words in S1 and S2 then:

      Jaccard (S1, S2) = |A ∩ B| / |A U B|

In [ ]:
a = [0, 1, 2, 5, 6, 8, 9]
b = [0, 2, 3, 4, 5, 7, 9]

In [ ]:
#Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
#Compute Jaccard Similarity
jaccard(a, b)

0.4

In [ ]:
#Jaccard Similarity between two sets of words
words_doc1 = {'data', 'are', 'valuable', 'assets', 'of', 'the', 'company'}
words_doc2 = {'data', 'are', 'assets'}

In [ ]:
#Compute Jaccard Similarity
jaccard(words_doc1, words_doc2)

0.42857142857142855

**5. Item-based** *Text Similarity (or Cosine Similarity)*

**TF-IDF**: *Term Frequency - Inverse Document Frequency*

The idea is to assign higher weights to words appearing frequently in a document (*TF weight*) and then to assign lower weights to words that appear frequently in the whole set of documents (*IDF weight*).

      1. We obtain vectors of weights (e.g., U and V for different documents)
      2. We then calculate the similarity using cosine similarity with these vectors.

In [ ]:
corpus = ['Data are valuable assets of the company',
          'Data quality is important for a company',
          'Data are assets',
          'Data Quality is important for data scientists',
          'Data are valuable products of the company']

In [ ]:
#Initialize an instance of count Vectorizer
count_vectorizer = CountVectorizer()
# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
#Generate the count matrix and the tf-idf vectors for the corpus
count_matrix = count_vectorizer.fit_transform(corpus)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [ ]:
#Extract the words
count_tokens = count_vectorizer.get_feature_names_out()
tfidf_tokens = tfidf_vectorizer.get_feature_names_out()

In [ ]:
#Create the dataframe with the count matrix and the tf-idf vectors
df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),index = ['Doc1','Doc2','Doc3','Doc4','Doc5'],columns = tfidf_tokens)
df_countvect = pd.DataFrame(data = count_matrix.toarray(),index = ['Doc1','Doc2','Doc3','Doc4','Doc5'],columns = count_tokens)

In [ ]:
print("Count Vectorizer\n")
print(df_countvect)
print("\nTF-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      are  assets  company  data  for  important  is  of  products  quality  \
Doc1    1       1        1     1    0          0   0   1         0        0   
Doc2    0       0        1     1    1          1   1   0         0        1   
Doc3    1       1        0     1    0          0   0   0         0        0   
Doc4    0       0        0     2    1          1   1   0         0        1   
Doc5    1       0        1     1    0          0   0   1         1        0   

      scientists  the  valuable  
Doc1           0    1         1  
Doc2           0    0         0  
Doc3           0    0         0  
Doc4           1    0         0  
Doc5           0    1         1  

TF-IDF Vectorizer

           are    assets   company      data       for  important        is  \
Doc1  0.346868  0.417868  0.346868  0.246800  0.000000   0.000000  0.000000   
Doc2  0.000000  0.000000  0.369830  0.263137  0.445529   0.445529  0.445529   
Doc3  0.581482  0.700505  0.000000  0.413729  

In [ ]:
#Compute and print the cosine similarity between the documents
cosine_sim = cosine_similarity(df_tfidfvect, df_tfidfvect)
cosine_sim

array([[1.        , 0.19322415, 0.59652436, 0.11072936, 0.78925825],
       [0.19322415, 1.        , 0.10886738, 0.79495102, 0.18476652],
       [0.59652436, 0.10886738, 1.        , 0.18562421, 0.29050797],
       [0.11072936, 0.79495102, 0.18562421, 1.        , 0.10588262],
       [0.78925825, 0.18476652, 0.29050797, 0.10588262, 1.        ]])

**PART 2: DUPLICATE DETECTION**

**Duplicate Detection** is the discovery of multiple representations of the same real-world object.
- Exact Matching
- Record Linkage (Not-Exact Matching)

**1. Exact Matching**

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/BEERS.csv")
data.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
...,...
2414,False
2415,False
2416,False
2417,False


In [ ]:
data.duplicated().any()

True

In [ ]:
print(data[data.duplicated()])

        abv   ibu    id                          name  \
1582   0.08  35.0     5                      Old Chub   
1606  55.00  55.0  1946   Overgrown American Pale Ale   
1676  57.00  68.0   711        Over the Rail Pale Ale   
1748  62.00   NaN  2373                   Oktoberfest   
1826   0.05  45.0  1324     Schlafly Yakima Wheat Ale   
1882   0.06  48.0   512  Autumnation (2011-12) (2011)   
2291  65.00  33.0  1932          Thai Style White IPA   
2340   0.07   NaN  2222                        10 Ton   
2406  45.00  25.0  1514               Easy Day Kolsch   

                        style brewery_id  ounces  
1582             Scottish Ale        166    12.0  
1606  American Pale Ale (APA)        261    12.0  
1676  American Pale Ale (APA)        135    12.0  
1748     Märzen / Oktoberfest         43    12.0  
1826  American Pale Wheat Ale        428    12.0  
1882              Pumpkin Ale         46    16.0  
2291       American White IPA         51    12.0  
2340            Oatme

In [ ]:
data.drop_duplicates()

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,66.000,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,71.000,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,75.000,NaN,2262,Sex and Candy,American IPA,177,12.0
...,...,...,...,...,...,...,...
2414,67.000,45.0,928,Belgorado,Belgian IPA,424,12.0
2415,0.052,NaN,807,Rail Yard Ale,American Amber / Red Ale,424,12.0
2416,55.000,NaN,620,B3K Black Lager,Schwarzbier,424,12.0
2417,55.000,40.0,145,Silverback Pale Ale,American Pale Ale (APA),424,12.0


**2. Record Linkage (RL)**

*Record Linkage* is the task of finding records in a data set that refer to the same entity (not-exact matching) across different data sources.

- Steps:
      1. Find the candidate records to compare
      2. Compare the candidate records and find duplicates

**RL** *Single Data Source*: Record linkage is a good option to find duplicates within a dataset.

**2.1 Find the candidate pairs**

The goal is to detect the record pairs to compare.

This process of making record pairs is also called ‘indexing’.

The ***recordlinkage*** library builds the set of pairs to compare.

In [ ]:
data1 = load_febrl1()
data1

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-188-dup-0,stephanie,geu,28,bainton crescent,masonic memorial village,maryborough,2541,sa,19421008,3997529
rec-334-dup-0,nicholas,NaN,289,britten-jonues drive,jabaru court,paddington,2000,vic,19970422,5062738
rec-469-dup-0,lachlan,katsiavos,29,paul coe cdrescent,NaN,casual,2913,nsw,19380406,4112327


In [ ]:
#First, load the recordlinkage.Index class and call the .full method.
#This object generates a full index on a .index(...) call.
#In case of deduplication of a single dataframe, one dataframe is sufficient as input argument.
indexer = recordlinkage.Index()

In [ ]:
indexer.full()
candidate_links = indexer.index(data1)

In [ ]:
#Numeber of comparison generated = (1000*1000-1000)/2 = 499500
print(len(data1), len(candidate_links))
candidate_links

1000 499500


MultiIndex([( 'rec-122-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-223-org'),
            ('rec-10-dup-0',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-373-org'),
            ( 'rec-227-org',   'rec-223-org'),
            ( 'rec-227-org',   'rec-122-org'),
            ( 'rec-227-org',   'rec-373-org'),
            ( 'rec-227-org',  'rec-10-dup-0'),
            ...
            ( 'rec-212-org',   'rec-205-org'),
            ( 'rec-212-org', 'rec-416-dup-0'),
            ( 'rec-212-org',   'rec-330-org'),
            ( 'rec-212-org',   'rec-385-org'),
            ( 'rec-212-org', 'rec-113-dup-0'),
            ( 'rec-212-org',   'rec-327-org'),
            ( 'rec-212-org', 'rec-188-dup-0'),
            ( 'rec-212-org', 'rec-334-dup-0'),
            ( 'rec-212-org', 'rec-469-dup-0'),
            ( 'rec-212-org', 'rec-350-dup-0')],
           names=['rec_id_1', 'rec_id_2'], 

**2.1.1 Find the candidate pairs** with *Blocking*

The file is partitioned in exclusive blocks and the comparison is limited to records within the same block.

***Blocking*** can be implemented by choosing a blocking key and grouping into a block all records that have the same values on the blocking key.

We cannot compare records from different blocks, this is a *huge limitation*.

In [ ]:
#Many of these record pairs do not belong to the same person.
#The recordlinkage toolkit has some more advanced indexing methods to reduce the number of record pairs.
#Obvious not-matches are left out of the index.
#Note that if a matching record pair is not included in the index, it can not be matched anymore
indexer = recordlinkage.Index()
indexer.block('given_name') # the blocks are builded on the basis of the provided attribute
candidate_links = indexer.index(data1)

In [ ]:
#The comparison will be limited to records with the same given_name
print(len(candidate_links))
candidate_links

2082


MultiIndex([(  'rec-251-org', 'rec-251-dup-0'),
            ('rec-284-dup-0',    'rec-67-org'),
            ('rec-335-dup-0',   'rec-335-org'),
            ( 'rec-23-dup-0',    'rec-23-org'),
            ( 'rec-32-dup-0',    'rec-70-org'),
            (   'rec-77-org', 'rec-382-dup-0'),
            ('rec-183-dup-0',   'rec-122-org'),
            ('rec-385-dup-0',    'rec-34-org'),
            (  'rec-302-org',    'rec-70-org'),
            (  'rec-302-org',  'rec-32-dup-0'),
            ...
            ('rec-469-dup-0', 'rec-183-dup-0'),
            ('rec-469-dup-0',   'rec-248-org'),
            ('rec-469-dup-0', 'rec-122-dup-0'),
            ('rec-469-dup-0',   'rec-469-org'),
            ('rec-469-dup-0',    'rec-74-org'),
            ('rec-469-dup-0',   'rec-183-org'),
            ('rec-469-dup-0', 'rec-360-dup-0'),
            ('rec-469-dup-0', 'rec-248-dup-0'),
            ('rec-350-dup-0',   'rec-350-org'),
            ('rec-350-dup-0',   'rec-242-org')],
           names=['rec_

In [ ]:
#Surname is discriminating more
indexer = recordlinkage.Index()
indexer.block('surname')
candidate_links = indexer.index(data1)

print(len(candidate_links))
candidate_links

1707


MultiIndex([(  'rec-335-org', 'rec-251-dup-0'),
            (  'rec-251-org', 'rec-251-dup-0'),
            (  'rec-251-org',   'rec-335-org'),
            ('rec-335-dup-0', 'rec-251-dup-0'),
            ('rec-335-dup-0',   'rec-335-org'),
            ('rec-335-dup-0',   'rec-251-org'),
            ('rec-320-dup-0', 'rec-305-dup-0'),
            (  'rec-167-org',   'rec-361-org'),
            ( 'rec-23-dup-0',    'rec-23-org'),
            (  'rec-419-org',   'rec-122-org'),
            ...
            (  'rec-327-org',    'rec-52-org'),
            (  'rec-327-org',   'rec-300-org'),
            (  'rec-327-org',   'rec-193-org'),
            (  'rec-327-org', 'rec-144-dup-0'),
            (  'rec-327-org', 'rec-300-dup-0'),
            (  'rec-327-org',   'rec-205-org'),
            ('rec-350-dup-0',   'rec-350-org'),
            (  'rec-212-org', 'rec-325-dup-0'),
            (  'rec-212-org',   'rec-325-org'),
            (  'rec-212-org', 'rec-212-dup-0')],
           names=['rec_

In [ ]:
#You can define the blocks among different attributes
indexer = recordlinkage.Index()
indexer.block('given_name', 'surname')
candidate_links = indexer.index(data1)

print(len(candidate_links))
candidate_links

197


MultiIndex([(  'rec-254-org', 'rec-254-dup-0'),
            (  'rec-418-org', 'rec-418-dup-0'),
            ('rec-149-dup-0',   'rec-149-org'),
            ('rec-149-dup-0',   'rec-418-org'),
            ('rec-239-dup-0',  'rec-85-dup-0'),
            ('rec-301-dup-0',   'rec-409-org'),
            ('rec-230-dup-0', 'rec-177-dup-0'),
            (  'rec-465-org', 'rec-301-dup-0'),
            (  'rec-306-org', 'rec-418-dup-0'),
            ( 'rec-63-dup-0', 'rec-301-dup-0'),
            ...
            (  'rec-322-org', 'rec-322-dup-0'),
            (   'rec-83-org', 'rec-213-dup-0'),
            (  'rec-220-org', 'rec-220-dup-0'),
            (  'rec-181-org', 'rec-376-dup-0'),
            ('rec-470-dup-0', 'rec-246-dup-0'),
            ('rec-437-dup-0', 'rec-273-dup-0'),
            (  'rec-340-org', 'rec-340-dup-0'),
            (  'rec-217-org', 'rec-340-dup-0'),
            ('rec-188-dup-0', 'rec-340-dup-0'),
            ('rec-469-dup-0',  'rec-74-dup-0')],
           names=['rec_

**2.1.2 Find the candidate pairs** with *Sorted Neighborhood*

***Sorted neighborhood*** consists of sorting a file and then moving a window of a fixed size on the file, comparing only records within the window.

In [ ]:
#First you have to sort the record in the database on the basis of a key (we use attributes to sort)
#Increasing the length of the window, the number of candidate links increases as well
indexer = recordlinkage.index.SortedNeighbourhood(
        on='given_name', window=9
    )

candidate_links = indexer.index(data1)
print(len(candidate_links))
candidate_links

11447


<ipython-input-43-0449b15e683b>:3: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  indexer = recordlinkage.index.SortedNeighbourhood(


MultiIndex([('rec-190-dup-0',   'rec-373-org'),
            (   'rec-73-org',   'rec-122-org'),
            (  'rec-186-org', 'rec-190-dup-0'),
            (  'rec-262-org',   'rec-452-org'),
            (  'rec-113-org',   'rec-409-org'),
            ('rec-291-dup-0',    'rec-23-org'),
            ('rec-291-dup-0',  'rec-23-dup-0'),
            (  'rec-102-org',   'rec-403-org'),
            (  'rec-312-org', 'rec-254-dup-0'),
            (   'rec-69-org',   'rec-350-org'),
            ...
            ('rec-469-dup-0',   'rec-279-org'),
            ('rec-469-dup-0',  'rec-73-dup-0'),
            ('rec-469-dup-0',   'rec-269-org'),
            ('rec-469-dup-0', 'rec-154-dup-0'),
            ('rec-469-dup-0',   'rec-159-org'),
            ('rec-350-dup-0',    'rec-69-org'),
            ('rec-350-dup-0', 'rec-258-dup-0'),
            ('rec-350-dup-0',  'rec-69-dup-0'),
            ('rec-350-dup-0',  'rec-66-dup-0'),
            ('rec-350-dup-0',    'rec-66-org')],
           names=['rec_

**2.2 Comparison between candidate pairs**

In [ ]:
#Compare records
compare_cl = recordlinkage.Compare()
#Define the way in which the tuples should be compared specifying the list of attributes
# these two attributes have to match exactly
compare_cl.exact('given_name', 'given_name', label='given_name')
#Here we compare attributes based on a similarity measure
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')

compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, data1)
features

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-190-dup-0 rec-373-org             0      0.0              0       0   
rec-73-org    rec-122-org             0      0.0              0       0   
rec-186-org   rec-190-dup-0           0      0.0              0       0   
rec-262-org   rec-452-org             0      0.0              0       0   
rec-113-org   rec-409-org             0      0.0              0       0   
...                                 ...      ...            ...     ...   
rec-350-dup-0 rec-69-org              0      0.0              0       0   
              rec-258-dup-0           0      0.0              0       0   
              rec-69-dup-0            0      0.0              0       0   
              rec-66-dup-0            0      0.0              0       0   
              rec-66-org              0      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-190-dup-0 rec-373-org        0        0.0  
rec-73-org    rec-122-org        0        0.0  
rec-186-org   rec-190-dup-0      1        0.0  
rec-262-org   rec-452-org        0        0.0  
rec-113-org   rec-409-org        0        0.0  
...                            ...        ...  
rec-350-dup-0 rec-69-org         0        0.0  
              rec-258-dup-0      0        0.0  
              rec-69-dup-0       0        0.0  
              rec-66-dup-0       0        0.0  
              rec-66-org         0        0.0  

[11447 rows x 6 columns]

In [ ]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,11447.000000,11447.000000,11447.000000,11447.000000,11447.000000,11447.000000
mean,0.181882,0.032934,0.029353,0.023762,0.253516,0.028566
std,0.385764,0.178473,0.168800,0.152312,0.435043,0.166592
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#The last step is to decide which records belong to the same person. In this example, we keep it simple:
features.sum(axis=1).value_counts().sort_index(ascending=False)

,count
6.0,142
5.0,167
4.0,53
3.0,15
2.0,388
1.0,3576
0.0,7106


In [ ]:
#Compare the matches
matches = features[features.sum(axis=1) > 5]

print(len(matches))
matches

142


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-335-dup-0,rec-335-org,1,1.0,1,1,1,1.0
rec-23-dup-0,rec-23-org,1,1.0,1,1,1,1.0
rec-452-dup-0,rec-452-org,1,1.0,1,1,1,1.0
rec-316-org,rec-316-dup-0,1,1.0,1,1,1,1.0
rec-77-dup-0,rec-77-org,1,1.0,1,1,1,1.0
...,...,...,...,...,...,...,...
rec-30-dup-0,rec-30-org,1,1.0,1,1,1,1.0
rec-138-org,rec-138-dup-0,1,1.0,1,1,1,1.0
rec-113-dup-0,rec-113-org,1,1.0,1,1,1,1.0


**RL** *Multiple Data Sources*

In [ ]:
hospital_accounts = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/HOSPITAL_ACC.csv")
hospital_reimbursement = pd.read_csv("https://raw.githubusercontent.com/camillasancricca/DATADIQ/master/HOSPITAL_REIM.csv")

In [ ]:
hospital_accounts

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
0,10605,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private
1,24250,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary
2,10341,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary
4,39835,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary
...,...,...,...,...,...,...,...,...,...,...
5334,92281,JEWISH HOSPITAL - SHELBYVILLE,727 HOSPITAL DRIVE,SHELBYVILLE,KY,40065,SHELBY,(502) 647-4300,Acute Care Hospitals,Voluntary non-profit - Private
5335,65248,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,MONTGOMERY,(937) 641-3450,Childrens,Voluntary non-profit - Private
5336,92377,NORTH TEXAS STATE HOSPITAL,6515 KEMP BLVD,WICHITA FALLS,TX,76308,WICHITA,(940) 692-1220,Psychiatric,Government - State
5337,71562,GULF BREEZE HOSPITAL,1110 GULF BREEZE PKWY,GULF BREEZE,FL,32561,SANTA ROSA,(850) 934-2000,Acute Care Hospitals,Voluntary non-profit - Other


In [ ]:
hospital_reimbursement

,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,519118,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,43,13289.09,5413.63,4490.93
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,73,22261.60,4922.18,4021.79
3,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
4,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53
...,...,...,...,...,...,...,...,...,...,...
2692,403227,BAYLOR SCOTT & WHITE MEDICAL CENTER- COLLEGE S...,700 SCOTT & WHITE DRIVE,COLLEGE STATION,TX,77845,37,20837.57,6389.76,3927.62
2693,797810,WALNUT HILL MEDICAL CENTER,7502 GREENVILLE AVENUE,DALLAS,TX,75231,52,32453.85,21927.08,4423.52
2694,611900,"BAY AREA REGIONAL MEDICAL CENTER, LLC",200 BLOSSOM STREET,WEBSTER,TX,77598,31,33006.84,19222.26,4497.16
2695,693781,RESOLUTE HEALTH HOSPITAL,"555 CREEKSIDE XING,",NEW BRAUNFELS,TX,78130,19,34370.00,9910.16,4531.37


1. ***Sorted Neighbourhood*** to find candidate pairs

In [ ]:
indexer = recordlinkage.index.SortedNeighbourhood(
    left_on='Facility Name', right_on='Provider Name', window=15
)

candidate_links = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidate_links))
candidate_links

38900


MultiIndex([(   0,  717),
            (   1,  516),
            (   2,  132),
            (   4, 2168),
            (   6, 1430),
            (  10, 2380),
            (  13, 2183),
            (  15, 2086),
            (  17, 1588),
            (  19, 2267),
            ...
            (5319, 2572),
            (5320,  528),
            (5323, 1173),
            (5324, 2010),
            (5325, 2649),
            (5328, 2070),
            (5332, 1777),
            (5333, 1167),
            (5336, 1172),
            (5338, 1256)],
           length=38900)

2. ***Compare*** the founded canditate pairs

In [ ]:
compare_cl = recordlinkage.Compare()

compare_cl.exact('Facility Name', 'Provider Name', label='name')
compare_cl.string('Address', 'Provider Street Address', method='jarowinkler', threshold=0.85, label='address')
compare_cl.string('City', 'Provider City', threshold=0.85, label='city')
compare_cl.exact('State', 'Provider State', label='state')
compare_cl.exact('ZIP Code', 'Provider Zip Code', label='address_1')

features = compare_cl.compute(candidate_links, hospital_accounts, hospital_reimbursement)
features

,,name,address,city,state,address_1
0,717,0,0.0,0.0,0,0
1,516,0,0.0,0.0,0,0
2,132,0,0.0,0.0,0,0
4,2168,0,0.0,0.0,0,0
6,1430,0,0.0,0.0,0,0
...,...,...,...,...,...,...
5328,2070,0,0.0,0.0,0,0
5332,1777,0,0.0,0.0,0,0
5333,1167,0,0.0,0.0,0,0
5336,1172,0,0.0,0.0,0,0


In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

,count
5.0,2240
4.0,129
3.0,48
2.0,280
1.0,5442
0.0,30761


In [ ]:
matches = features[features.sum(axis=1) > 4]

print(len(matches))
matches

2240


,,name,address,city,state,address_1
14,1777,1,1.0,1.0,1,1
27,1502,1,1.0,1.0,1,1
28,779,1,1.0,1.0,1,1
34,2170,1,1.0,1.0,1,1
38,1308,1,1.0,1.0,1,1
...,...,...,...,...,...,...
5326,2400,1,1.0,1.0,1,1
5328,65,1,1.0,1.0,1,1
5329,2213,1,1.0,1.0,1,1
5334,988,1,1.0,1.0,1,1


**Summary**

*Edit Distance/Levenshtein Distance*
- Levenshtein.distance()
- Levenshtein.ratio()

*Sounded Code (American Soundex)*
- Soundex().phonetic()
- Soundex().sounds_like()
- Soundex().distance()

*Jaro-Winkler*
- jaro.jaro_winkler_metric


*TF-IDF (sklearn)*
- CountVectorizer()
- TfidfVectorizer()
- cosine_similarity()

*Exact Duplicates*
- DataFrame.duplicated()
- DataFrame.drop_duplicates()

*Record Linkage*
- recordlinkage.Index().block()
- recordlinkage.Index().SortedNeighbourhood()
- recordlinkage.Index().index()
- recordlinkage.Compare().compute()